In [ ]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
from PIL import ImageGrab
import datetime
import asyncio
from queue import Queue
import nest_asyncio
import io
from mss import mss
from PIL import Image, ImageDraw
import pyautogui
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None

async def generate_frames():
    global latest_frame
    # Variable for frame interval (in seconds) to achieve 1 FPS
    frame_interval = 1
    start_time = time.time()
    num_frames = 0
    while True:
        frame_start_time = time.time()
        img = ImageGrab.grab()  # Capture the screen
        img = np.array(img)
        img_final = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if True:
            img_final = cv2.resize(img_final, (1920, 1080)) 
            _, buffer = cv2.imencode('.jpg', img_final)
            num_frames += 1

            latest_frame = buffer.tobytes()  # Update the latest frame

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # If one second has elapsed, calculate and print FPS
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time
            # print("FPS:", fps)
            num_frames = 0
            start_time = time.time()

        frame_time = time.time() - frame_start_time
        # Introduce the delay to achieve 1 FPS
        delay = frame_interval - frame_time
        try:
            await asyncio.sleep(delay)
        except asyncio.CancelledError:
            break


# Load and resize the mouse icon once
mouse_icon = Image.open("mouse_icon.png")
n = 26
new_size = (n, n)
mouse_icon = mouse_icon.resize(new_size, Image.NEAREST)
icon_width, icon_height = mouse_icon.size

# Create an mss instance
sct = mss()


def capture_screen_with_mouse():
    s = time.time()
    
    # Get the current mouse cursor position
    mouse_x, mouse_y = pyautogui.position()
    position = [mouse_x, mouse_y]
    print(f"Mouse position: ({mouse_x}, {mouse_y})")
    
    # Capture the screen
    monitor = {"top": 0, "left": 0, "width": 1920, "height": 1080}  # Adjust these values to match your screen resolution
    sct_img = sct.grab(monitor)
    screenshot = Image.frombytes("RGB", (sct_img.width, sct_img.height), sct_img.rgb)
    
    # Calculate the position to paste the mouse icon
    icon_position = (mouse_x - icon_width // 2, mouse_y - icon_height // 2)
    
    # Create a drawing context
    draw = ImageDraw.Draw(screenshot)
    
    # Draw the inner blue rectangle around the mouse icon
    box_padding = 2  # Adjust this value to change the thickness of the box
    inner_box_position = (
        icon_position[0] - box_padding,
        icon_position[1] - box_padding,
        icon_position[0] + icon_width + box_padding,
        icon_position[1] + icon_height + box_padding
    )
    # draw.rectangle(inner_box_position, outline="blue", width=box_padding)
    
    # Draw the outer rectangle 30 pixels away from the inner rectangle
    outer_padding = 30
    outer_box_position = (
        inner_box_position[0] - outer_padding,
        inner_box_position[1] - outer_padding,
        inner_box_position[2] + outer_padding,
        inner_box_position[3] + outer_padding
    )
    draw.rectangle(outer_box_position, outline="red", width=box_padding)
    
    # Paste the mouse icon onto the screenshot
    screenshot.paste(mouse_icon, icon_position, mouse_icon)
    
    e = time.time()
    r = e - s
    print(f'total {r}')
    
    return screenshot,position


@app.get("/screenshot")
async def get_screenshot():
    screenshot, position = capture_screen_with_mouse()
    
    # Convert the screenshot to bytes
    img_byte_arr = io.BytesIO()
    screenshot.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    
    # Prepare the response headers
    headers = {
        "Content-Disposition": "attachment; filename=screenshot.png",
        "Mouse-Position-X": str(position[0]),
        "Mouse-Position-Y": str(position[1])
    }
    
    # Return the image as a streaming response
    return StreamingResponse(io.BytesIO(img_byte_arr), media_type="image/png", headers=headers)

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global latest_frame
        while True:
            start_time = time.time()  # Record start time of sending frames
            for _ in range(1):  # Send 1 frame per second
                if latest_frame is not None:  # Check if there's a latest frame available
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )
                else:
                    await asyncio.sleep(0.15)  # If no latest frame available, wait for a short time

            # Calculate elapsed time and wait if necessary to maintain 1 FPS rate
            elapsed_time = time.time() - start_time
            if elapsed_time < 1:
                await asyncio.sleep(1 - elapsed_time)

    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")




async def main():
    # Start generating frames asynchronously
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())

    
    uvicorn.run(app, host="localhost", port=3000)

INFO:     Started server process [448]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:3000 (Press CTRL+C to quit)


INFO:     27.147.206.228:0 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     34.121.142.92:0 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     34.121.142.92:0 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     34.121.142.92:0 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     34.121.142.92:0 - "GET /video_feed HTTP/1.1" 200 OK
